In [47]:
import pandas as pd
import openpyxl
import time
import numpy as np

In [48]:
def combine(x):
    # Функция для группировки всех значений в строку разделенную ;
    return  ';'.join(x)

In [49]:
def check_unique(x):
    # Функция для нахождения разночтений в площади выделенного гектара
    temp_lst = x.split(';')
    temp_set = set(temp_lst)
    if'nan' in temp_set:
        return 'Не заполнены значения площади лесотаксационного выдела!!!'
    else:
        return 'Площади совпадают' if len(temp_set) == 1 else 'Ошибка!!! Площади не совпадают'

In [50]:
df = pd.read_excel('data/BLPK.xlsx',sheet_name='Реестр УПП',skiprows=6)

In [51]:
# Удаляем лишние строки
df = df.drop([0,1],axis=0)

In [52]:
df.shape

(127014, 27)

In [53]:
# СОздаем проверочный файл для проверки правильности ввода плошади выдела
check_df = df.copy()

In [54]:
# Меняем тип столбца на строку чтобы создать строку включающую в себя все значения разделенные ;
check_df['Площадь лесотаксационного выдела, га'] = check_df['Площадь лесотаксационного выдела, га'].astype(str)

checked_pl=check_df.groupby(['Лесничество','Участковое лесничество','Урочище ','Номер лесного квартала','Номер лесотаксационного выдела']).agg({'Площадь лесотаксационного выдела, га':combine})

# Применяем функцию првоеряющую количество уникальных значений в столбце, если больше одного то значит есть ошибка в данных
checked_pl['Контроль совпадения площади выдела'] = checked_pl['Площадь лесотаксационного выдела, га'].apply(check_unique)


# переименовывам колонку
checked_pl.rename(columns={'Площадь лесотаксационного выдела, га':'Все значения площади для указанного выдела'},inplace=True)
#Извлекаем индексы в колонки
checked_pl = checked_pl.reset_index()
# Заполняем nan в колонке со значениями площади
#checked_pl['Контроль совпадения площади выдела'] = checked_pl['Все значения площади для указанного выдела'].apply(lambda x:'Не заполнены значения площади!!!' if x == 'nan' else x)

#Получаем текущую дату
current_time = time.strftime('%d.%m.%Y')
#Сохраняем отчет

checked_pl.to_excel(f'Отчет Контроль совпадения площади выдела от {current_time}.xlsx',index=False)

In [55]:
# Готовим колонки к группировке
df['Площадь лесотаксационного выдела, га'] = df['Площадь лесотаксационного выдела, га'].astype(str)

df['Площадь лесотаксационного выдела, га'] = df['Площадь лесотаксационного выдела, га'].apply(lambda x:x.replace(',','.'))

df['Площадь лесотаксационного выдела, га'] = df['Площадь лесотаксационного выдела, га'].astype(float)

df['Площадь лесотаксационного выдела или его части (лесопатологического выдела), га'] = df['Площадь лесотаксационного выдела или его части (лесопатологического выдела), га'].astype(str)

df['Площадь лесотаксационного выдела или его части (лесопатологического выдела), га'] = df['Площадь лесотаксационного выдела или его части (лесопатологического выдела), га'].apply(lambda x:x.replace(',','.'))

df['Площадь лесотаксационного выдела или его части (лесопатологического выдела), га'] = df['Площадь лесотаксационного выдела или его части (лесопатологического выдела), га'].astype(float)

In [56]:
#Группируем
group_df=df.groupby(['Лесничество','Участковое лесничество','Урочище ','Номер лесного квартала','Номер лесотаксационного выдела']).agg({'Площадь лесотаксационного выдела, га':'sum','Площадь лесотаксационного выдела или его части (лесопатологического выдела), га':'sum'})

In [58]:
group_df.rename(columns={'Площадь лесотаксационного выдела или его части (лесопатологического выдела), га':'Используемая площадь лесотаксационного выдела, га'},inplace=True)

In [59]:
# Извлекаем индексы в колонки
group_df = group_df.reset_index()

In [60]:
group_df['Площадь лесотаксационного выдела, га']=group_df['Площадь лесотаксационного выдела, га'].astype(float)
group_df['Используемая площадь лесотаксационного выдела, га']=group_df['Используемая площадь лесотаксационного выдела, га'].astype(float)

In [61]:
group_df['Контроль площади используемого надела'] = group_df['Площадь лесотаксационного выдела, га'] < group_df['Используемая площадь лесотаксационного выдела, га']

In [62]:
group_df['Контроль площади используемого надела'] = group_df['Контроль площади используемого надела'].apply(lambda x:'Превышение используемой площади выдела!!!' if x is True else 'Все в порядке')

In [63]:
group_df.to_excel(f'Отчет о площадях выделов от {current_time}.xlsx',index=False)